This is going to be a blog about fun random ideas I come up with hyped on coffee. As my first post I'll be exploring the idea making a motion sensor using OpenCV and Python. The purpose is to detect ambient motion in a relatively still room. The motion sensor will keep rolling average of motion levels present and make some kind of notification when motion spikes.  

I'll be using the OpenCV background subtractor to detect motion levels. 

https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html

In [1]:
import numpy as np
import cv2

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg =  cv2.createBackgroundSubtractorMOG2()


In a perfectly still frame the background subtractor would return only black pixels. To determine the amount of a motion in a frame we need only sum up all the pixels in a frame and determine if the number we get is unusually high. 

To do this I've set up a moving window to collect stats on motion levels and set the window size to 100 frames. For determining if a value is unusually high, I'll look for values 1 standard deviation over the mean.


In [ ]:
moving_window_array = np.array([])

window_size = 100

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    fgmask_sum = fgmask.sum()
        
    moving_window_array = np.append(moving_window_array, fgmask_sum)

    #Waits until there are enough frames to evaluate the relative motion levels. 
    if len(moving_window_array) > window_size : 
    
        moving_window_array = moving_window_array[: window_size]
        
        if  fgmask_sum > moving_window_array.mean() + moving_window_array.std(): 
            text = "Hello there."
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, text, (50, 50), font, 2, (255, 255, 0), 2)

    
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

By tuning the parameters one can increase the moving average window or lower the threshold for what constitutes a motion spike. 